## Stable Diffusion

In [7]:
! python -m pip install --upgrade pip
! pip install botocore --upgrade --quiet

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached pip-23.0.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1


In [8]:
!pip list|grep -i diffuser

diffusers                     0.14.0


In [8]:
!pip install "sagemaker==2.116.0" "huggingface_hub==0.13.1" --upgrade --quiet

In [9]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::687912291502:role/service-role/AmazonSageMaker-ExecutionRole-20211013T113123
sagemaker bucket: sagemaker-us-west-2-687912291502
sagemaker session region: us-west-2


In [10]:
#!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install -r ./requirements.txt
!torch_version=1.13.1
!pip3 install torch==1.13.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!pip3 install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.4/716.4 kB 4.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 19.7 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.5/763.5 kB 5.2 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.7/18.7 MB 26.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 21.1 MB/s eta

In [27]:
import os
import json
from diffusers import StableDiffusionPipeline
from diffusers import StableDiffusionImg2ImgPipeline
import boto3
import sagemaker
import uuid
import torch
from torch import autocast
from PIL import Image
import io
import requests
import traceback
import os
import json
import torch
from diffusers import StableDiffusionPipeline
from diffusers import StableDiffusionImg2ImgPipeline
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import deepspeed



#model_dir='/root/dreambooth/models_diffuser/'
#model_dir='runwayml/stable-diffusion-v1-5'
#model_dir='andite/anything-v4.0'
model_dir='/home/ec2-user/SageMaker/dst/'
model = StableDiffusionPipeline.from_pretrained(model_dir, torch_dtype=torch.float16, revision="fp16")
#model = StableDiffusionPipeline.from_pretrained(model_dir, torch_dtype=torch.float16, revision="fp16")
print("model loaded:",model)
 
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True


#try:
#   print("begin load deepspeed....")    
#   model=deepspeed.init_inference(
#       model=getattr(model,"model", model),      # Transformers models
#       mp_size=1,        # Number of GPU
#       dtype=torch.float16, # dtype of the weights (fp16)
#       replace_method="auto", # Lets DS autmatically identify the layer to replace
#       replace_with_kernel_inject=False, # replace the model with the kernel injector
#   )
#   print('model accelarate with deepspeed!')
#except Exception as e:
#   print("deepspeed accelarate excpetion!")
#   print(e)
    
model = model.to("cuda")
model.enable_attention_slicing()    

model loaded: StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.13.1",
  "feature_extractor": [
    "transformers",
    "CLIPFeatureExtractor"
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}



### Inference

In [28]:
from PIL import Image
from io import BytesIO
from IPython.display import display
import base64
import matplotlib.pyplot as plt

# helper decoder
def decode_base64_image(image_string):
  base64_image = base64.b64decode(image_string)
  buffer = BytesIO(base64_image)
  return Image.open(buffer)

# display PIL images as grid
def display_images(images=None,columns=3, width=100, height=100):
    plt.figure(figsize=(width, height))
    for i, image in enumerate(images):
        plt.subplot(int(len(images) / columns + 1), columns, i + 1)
        plt.axis('off')
        plt.imshow(image)
        
def get_bucket_and_key(s3uri):
    pos = s3uri.find('/', 5)
    bucket = s3uri[5 : pos]
    key = s3uri[pos + 1 : ]
    return bucket, key        

In [30]:
%%time
s3_client = boto3.client('s3')
#prompt = "A dog trying catch a flying pizza art drawn by disney concept artists, golden colour, high quality, highly detailed, elegant, sharp focus"
prompt = "portrait photo headshot by mucha, sharp focus, elegant, render, octane, detailed, award winning photography, masterpiece, rim lit"
#prompt = "priest, blue robes, 68 year old nun, national geographic, portrait, photo, photography"
#prompt = "hotel room with a swimming pool outside of the window, TV on the table, moon in the sky"


images = model(prompt, num_images_per_prompt=5, num_inference_steps=50, guidance_scale=9).images
ret_images=[]
for image in images:
   bucket, key = get_bucket_and_key("s3://sagemaker-us-west-2-687912291502/deepspeed/")
   key = '{0}{1}.jpg'.format(key, uuid.uuid4())
   buf = io.BytesIO()
   image.save(buf, format='JPEG')
   s3_client.put_object(
       Body = buf.getvalue(), 
       Bucket = bucket, 
       Key = key, 
       ContentType = 'image/jpeg'
   )
   ret_images.append(image)
   print('image: ', 's3://{0}/{1}'.format(bucket, key))



  0%|          | 0/50 [00:00<?, ?it/s]

image:  s3://sagemaker-us-west-2-687912291502/deepspeed/c49f69ac-12aa-4963-88db-84b964af7d7c.jpg
image:  s3://sagemaker-us-west-2-687912291502/deepspeed/32f11ffd-2003-47e3-8f4b-f910aef81c2d.jpg
image:  s3://sagemaker-us-west-2-687912291502/deepspeed/77b2f4ac-7aad-4260-83cd-18469ad60d73.jpg
image:  s3://sagemaker-us-west-2-687912291502/deepspeed/a8415586-fbd8-4998-a630-571fe88fa092.jpg
image:  s3://sagemaker-us-west-2-687912291502/deepspeed/b5626bea-74a7-4016-8cf2-55300ae0494b.jpg
CPU times: user 15.6 s, sys: 3.33 s, total: 18.9 s
Wall time: 19.2 s


In [31]:
# visualize generation
display_images(ret_images)